<a href="https://colab.research.google.com/github/jsosorioe/curso-python-data-structures/blob/main/Cursera_data_bases_python_week_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Carga los datos a Colab

!wget  http://www.py4e.com/code3/tracks.zip
!unzip tracks.zip



--2022-05-13 22:22:19--  http://www.py4e.com/code3/tracks.zip
Resolving www.py4e.com (www.py4e.com)... 104.21.5.179, 172.67.133.177, 2606:4700:3034::ac43:85b1, ...
Connecting to www.py4e.com (www.py4e.com)|104.21.5.179|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.py4e.com/code3/tracks.zip [following]
--2022-05-13 22:22:19--  https://www.py4e.com/code3/tracks.zip
Connecting to www.py4e.com (www.py4e.com)|104.21.5.179|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52332 (51K) [application/zip]
Saving to: ‘tracks.zip’

tracks.zip          100%[===================>]  51.11K  --.-KB/s    in 0s      

2022-05-13 22:22:19 (122 MB/s) - ‘tracks.zip’ saved [52332/52332]

Archive:  tracks.zip
   creating: tracks/
  inflating: tracks/tracks.py        
  inflating: tracks/README.txt       
  inflating: tracks/Library.xml      


In [ ]:
import xml.etree.ElementTree as ET
import sqlite3

conn = sqlite3.connect('trackdb.sqlite')
cur = conn.cursor()

# Make some fresh tables using executescript()
cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;

CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
''')


fname = input('Enter file name: ')
if ( len(fname) < 1 ) : fname = 'Library.xml'

# <key>Track ID</key><integer>369</integer>
# <key>Name</key><string>Another One Bites The Dust</string>
# <key>Artist</key><string>Queen</string>
def lookup(d, key):
    found = False
    for child in d:
        if found : return child.text
        if child.tag == 'key' and child.text == key :
            found = True
    return None

stuff = ET.parse(fname)
all = stuff.findall('dict/dict/dict')
print('Dict count:', len(all))
for entry in all:
    if ( lookup(entry, 'Track ID') is None ) : continue

    name = lookup(entry, 'Name')
    artist = lookup(entry, 'Artist')
    album = lookup(entry, 'Album')
    count = lookup(entry, 'Play Count')
    rating = lookup(entry, 'Rating')
    length = lookup(entry, 'Total Time')

    if name is None or artist is None or album is None : 
        continue

    print(name, artist, album, count, rating, length)

    cur.execute('''INSERT OR IGNORE INTO Artist (name) 
        VALUES ( ? )''', ( artist, ) )
    cur.execute('SELECT id FROM Artist WHERE name = ? ', (artist, ))
    artist_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id) 
        VALUES ( ?, ? )''', ( album, artist_id ) )
    cur.execute('SELECT id FROM Album WHERE title = ? ', (album, ))
    album_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Track
        (title, album_id, len, rating, count) 
        VALUES ( ?, ?, ?, ?, ? )''', 
        ( name, album_id, length, rating, count ) )

    conn.commit()